In [ ]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.0 MB/s eta 0:00:00


### Built-in Tool-DuckDuckGo Search

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke('ipl news')
print(results)

IPL 2025 suspension Highlights: The Indian Premier League has been suspended with immediate effect for one week as India is engaged in a conflict with Pakistan. The decision on new schedule and venues will come soon. The Board of Control for Cricket in India suspended the Indian Premier League 2025 season for a week amid rising cross-border tensions between India and Pakistan. On Thursday, May 8 the IPL match ... BCCI Suspends IPL 2025 LIVE: The 2025 Indian Premier League (IPL) has been suspended for a week with immediate effect due to escalating military tensions between India and Pakistan, the BCCI ... IPL 2025 Suspended LIVE Updates: The BCCI has suspended the ongoing IPL 2025 season indefinitely, due to escalating Indo-Pak tensions. IPL 2025 Highlights: A cloud of uncertainty had loomed over the future of the ongoing edition since the cancellation of Thursday's match between Punjab Kings and Delhi Capitals.


### Built-in Shell  Tool

In [2]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()
results = shell_tool.invoke('whoami')
print(results)

Executing command:
 whoami
desktop-kjhp78p\dell



d:\Langchain Model\venv\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [3]:
from langchain_core.tools import tool


In [4]:
#step 1 - create a function

def multiply(a,b):
  """Multiply 2 numbers"""
  return a*b

In [5]:
#Step 2 - add type hints

def multiply(a: int, b: int) -> int:
  """Multiply 2 numbers"""
  return a*b



In [6]:
#step 3 - add tool decorator

@tool
def multiply(a: int, b: int) -> int:
  """Multiply 2 numbers"""
  return a*b

  # with tool - decorator LLM will be able to communicate with the multiply function


In [7]:
result = multiply.invoke({'a':2,'b':3})
print(result)

6


In [8]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply 2 numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [9]:
# The following is given to LLM
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply 2 numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


### Method 2 - Using StructuredTool

In [10]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [11]:
class MultiplyInput(BaseModel):
  a: int = Field(required=True,description='First numberto add')
  b: int = Field(required=True,description='Second number to add')

In [12]:
def multiply_func(a:int, b:int) -> int:
  return a*b

In [13]:
multiply_tool = StructuredTool.from_function(
    func= multiply_func,
    name = 'Multiply',
    description = 'Multiply 2 numbers',
    args_schema = MultiplyInput #pydantic class
)

In [14]:
result = multiply_tool.invoke({'a':2,'b':3})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

6
Multiply
Multiply 2 numbers
{'a': {'description': 'First numberto add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


In [15]:
### Method 3 - Using BaseTool Class

In [16]:
from langchain.tools import BaseTool
from typing import Type

In [17]:
#arg schema using pydantic

class MultiplyInput(BaseModel):
  a: int = Field(required=True,description='First numberto add')
  b: int = Field(required=True,description='Second number to add')

In [18]:
class MultiplyTool(BaseTool):
  name: str = "Multiply"
  description: str = "Multiply 2 numbers"

  args_schema: Type[BaseModel] = MultiplyInput

  def _run(self, a: int, b: int) -> int:
    return a*b

In [19]:
multiply_tool = MultiplyTool()

In [20]:
result = multiply_tool.invoke({'a':2,'b':3})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

6
Multiply
Multiply 2 numbers
{'a': {'description': 'First numberto add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### ToolKit

In [21]:
from langchain.tools import tool

#custom Tools

@tool
def add(a: int, b: int) -> int:
  """ Add two numbers"""
  return a+b

@tool
def multiply(a: int, b: int) -> int:
  """Multiply 2 numbers"""
  return a*b

In [22]:
class MathToolkit:
  def get_tools(self):
    return [add,multiply]

In [23]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
  print(tool.name, "=>",tool.description)

add => Add two numbers
multiply => Multiply 2 numbers


### Tool Binding

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [4]:
#tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product: """
  return a*b

In [5]:
print(multiply.invoke({'a':2,'b':3}))

6


### Tool Binding

In [6]:
llm = ChatOpenAI()

In [7]:
llm_with_tools = llm.bind_tools([multiply])
#llm_with_tools is a llm that can use the multiply tool

### Tool Calling

In [8]:
llm_with_tools.invoke('Hi How are you?')

AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with any questions or tasks you have. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 59, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BVckEEfd3Y7Ubf2CrMXPEgwMNnjFN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8f611c24-1a1f-477d-9764-331ff4796bc9-0', usage_metadata={'input_tokens': 59, 'output_tokens': 40, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
query = HumanMessage("can you multiply 2 and 3?")
messages = [query]

In [ ]:
result = llm_with_tools.invoke(messages)

In [18]:
messages.append(result)
print(messages)

[HumanMessage(content='can you multiply 2 and 3?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0YsWA1e7TMFhgNGDi1ExgH1H', 'function': {'arguments': '{"a": 2, "b": 3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 63, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BVctGLLQMqd6mlGWdiFoL5bbNi1fw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--fd457906-8a8e-4dc5-a854-538028b4a661-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_0YsWA1e7TMFhgNGDi1ExgH1H', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63

### Tool Execution

In [15]:
# this will send the arguments to the multiply tool
multiply.invoke(result.tool_calls[0]['args'])

6

In [19]:
#This will sent the tool call schema to the multiply tool 
#This will return tool message 
tool_result = multiply.invoke(result.tool_calls[0])
messages.append(tool_result)


In [22]:
# conversation History - HumanMessage, AIMessage, SystemMessage, ToolMessage
messages

[HumanMessage(content='can you multiply 2 and 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0YsWA1e7TMFhgNGDi1ExgH1H', 'function': {'arguments': '{"a": 2, "b": 3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 63, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BVctGLLQMqd6mlGWdiFoL5bbNi1fw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--fd457906-8a8e-4dc5-a854-538028b4a661-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_0YsWA1e7TMFhgNGDi1ExgH1H', 'type': 'tool_call'}], usage_metadata={'input_tokens': 6

In [ ]:
llm_with_tools.invoke(messages)